## Lista 5

In [1]:
from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
import numpy as np
import networkx as nx
import pandas as pd
from graphframes import *
import sys
import json

In [2]:
config = SparkConf().setAppName('appName').setMaster('local')
sctext = SparkContext.getOrCreate(config)
spark = SparkSession(sctext)

In [3]:
from graphframes import *

No = spark.read.csv('transport-nodes.csv', header=True)
relation = spark.read.csv('transport-relationships.csv', header=True)
graph = GraphFrame(No, relation)

### Crie um CSV para armazenar as cidades, com id (nome da cidade), latitude, longitude e população.

In [7]:
No.show()

+--------------------+---------+---------+----------+
|                  id| latitude|longitude|population|
+--------------------+---------+---------+----------+
|         João Pessoa|52.379189| 4.899431|    809015|
|      Campina Grande|52.092876| 5.104480|    334176|
|               Patos|52.078663| 4.288788|    107605|
|            Cabedelo| 53.61239| -0.22219|      9642|
|           São Bento| 53.52285| -1.13116|     34031|
|             Triunfo|  51.9775|  4.13333|      9382|
|          Mamanguape| 51.96375|   1.3511|     44882|
|          Itaporanga| 52.05917|  1.15545|     24692|
|               Areia| 51.88921|  0.90421|     22819|
|           Rio Tinto|51.509865|-0.118092|     24172|
|São José do rio d...|  51.9225|  4.47917|     18034|
|          São Mamede| 52.01667|  4.70833|      7724|
+--------------------+---------+---------+----------+



### Crie outro CSV para armazenar a distância entre essas cidades, com src, dst e relationship como campos (adicione pelo menos 30 registros nesse arquivo).

In [8]:
relation.show()

+--------------------+--------------------+------------+----+
|                 src|                 dst|relationship|cost|
+--------------------+--------------------+------------+----+
|         João Pessoa|      Campina Grande|       EROAD|  42|
|         João Pessoa|               Patos|       EROAD|  56|
|               Patos|            Cabedelo|       EROAD|  19|
|         João Pessoa|           São Bento|       EROAD| 297|
|           São Bento|          São Mamede|       EROAD|  35|
|          São Mamede|             Triunfo|       EROAD| 209|
|          Mamanguape|               Patos|       EROAD|  17|
|               Areia|          Mamanguape|       EROAD|  75|
|          Itaporanga|               Areia|       EROAD| 132|
|           Rio Tinto|          Itaporanga|       EROAD|  43|
|             Triunfo|           Rio Tinto|       EROAD|  85|
|São José do rio d...|            Cabedelo|       EROAD|  23|
|São José do rio d...|      Campina Grande|       EROAD|  32|
|       

## Utilizando as bibliotecas do Spark, crie um objeto GraphFrame a partir desses dois CSVs.

In [9]:
# Utilizando as bibliotecas do Spark, crie um objeto GraphFrame a partir desses dois CSVs.

# Número de relações entre os nós
graph.degrees.show()

+--------------------+------+
|                  id|degree|
+--------------------+------+
|               Areia|     3|
|            Cabedelo|     4|
|           São Bento|     3|
|          São Mamede|     4|
|         Itaporanga |     1|
|             Triunfo|     2|
|         João Pessoa|     6|
|           Rio Tinto|     3|
|São José do rio d...|     5|
|               Patos|     5|
|      Campina Grande|     4|
|          Mamanguape|     3|
|          Itaporanga|     5|
+--------------------+------+



## Utilizando o método bfs (Breadth First Search), execute 5 filtragens a sua escolha.

In [10]:
# Método 1

graph.bfs(fromExpr = "id = 'Areia'", toExpr = "id = 'Itaporanga'", edgeFilter = "relationship = 'EROAD'").show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                from|                  e0|                  v1|                  e1|                  v2|                  e2|                  v3|                  e3|                  v4|                  e4|                  v5|                  e5|                  v6|                  e6|                  to|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|[Areia, 51.88921,...|[Areia, Mamanguap...|[Maman

In [126]:
# Método 2

graph.bfs(fromExpr = "id = 'Triunfo'", toExpr = "id = 'Patos'", edgeFilter = "relationship = 'EROAD'").show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                from|                  e0|                  v1|                  e1|                  v2|                  e2|                  v3|                  e3|                  v4|                  e4|                  to|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|[Triunfo, 51.9775...|[Triunfo, Rio Tin...|[Rio Tinto, 51.50...|[Rio Tinto, Itapo...|[Itaporanga, 52.0...|[Itaporanga, Arei...|[Areia, 51.88921,...|[Areia, Mamanguap...|[Mamanguape, 51.9...|[Mamanguape, Pato...|[Patos, 52.078663...|
+--------------------+--------------------+--------------------+----

In [127]:
# Método 3

graph.bfs(fromExpr = "id = 'João Pessoa'", toExpr = "id = 'Cabedelo'", edgeFilter = "relationship = 'EROAD'").show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                from|                  e0|                  v1|                  e1|                  to|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|[João Pessoa, 52....|[João Pessoa, São...|[São José do rio ...|[São José do rio ...|[Cabedelo, 53.612...|
|[João Pessoa, 52....|[João Pessoa, Pat...|[Patos, 52.078663...|[Patos, Cabedelo,...|[Cabedelo, 53.612...|
+--------------------+--------------------+--------------------+--------------------+--------------------+



In [128]:
# Método 4

graph.bfs(fromExpr = "id = 'Campina Grande'", toExpr = "id = 'Mamanguape'", edgeFilter = "relationship = 'EROAD'").show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                from|                  e0|                  v1|                  e1|                  v2|                  e2|                  v3|                  e3|                  v4|                  e4|                  to|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|[Campina Grande, ...|[Campina Grande, ...|[João Pessoa, 52....|[João Pessoa, Rio...|[Rio Tinto, 51.50...|[Rio Tinto, Itapo...|[Itaporanga, 52.0...|[Itaporanga, Arei...|[Areia, 51.88921,...|[Areia, Mamanguap...|[Mamanguape, 51.9...|
+--------------------+--------------------+--------------------+----

In [129]:
# Método 5

graph.bfs(fromExpr = "id = 'Rio Tinto'", toExpr = "id = 'Areia'", edgeFilter = "relationship = 'EROAD'").show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                from|                  e0|                  v1|                  e1|                  to|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|[Rio Tinto, 51.50...|[Rio Tinto, Itapo...|[Itaporanga, 52.0...|[Itaporanga, Arei...|[Areia, 51.88921,...|
+--------------------+--------------------+--------------------+--------------------+--------------------+



## Execute 2 consultas utilizando o método find.

In [130]:
# Método Find 1)
graph.find("(a)-[relation]->(b); (a)-[relation2]->(c); (b)-[relation3]->(c)").show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                   a|            relation|                   b|           relation2|                   c|           relation3|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|[João Pessoa, 52....|[João Pessoa, Pat...|[Patos, 52.078663...|[João Pessoa, São...|[São José do rio ...|[Patos, São José ...|
|[João Pessoa, 52....|[João Pessoa, Pat...|[Patos, 52.078663...|[João Pessoa, Cam...|[Campina Grande, ...|[Patos, Campina G...|
|[Patos, 52.078663...|[Patos, São José ...|[São José do rio ...|[Patos, Campina G...|[Campina Grande, ...|[São José do rio ...|
|[Patos, 52.078663...|[Patos, São José ...|[São José do rio ...|[Patos, Cabedelo,...|[Cabedelo, 53.612...|[São José do rio ...|
|[João Pessoa, 52....|[João Pessoa, São...|[São José do rio ...|[João Pessoa, Cam...|[Campina Grande, ..

In [131]:
# Execute 2 consultas utilizando o método find.

# Método find 2)
graph.find("(a)-[relation]->(b); (a)-[relation2]->(c); (a)-[relation3]->(d); (a)-[relation4]->(e)").show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                   a|            relation|                   b|           relation2|                   c|           relation3|                   d|           relation4|                   e|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|[João Pessoa, 52....|[João Pessoa, Cam...|[Campina Grande, ...|[João Pessoa, São...|[São José do rio ...|[João Pessoa, São...|[São José do rio ...|[João Pessoa, São...|[São José do rio ...|
|[João Pessoa, 52....|[João Pessoa, Cam...|[Campina Grande, ...|[João Pessoa, São...|[São José do rio ...|[João Pessoa, São...|[São José do rio ...|[João Pessoa, Rio...|[Rio Tinto, 51.50...|
|[João Pessoa, 52....|[João Pessoa, Cam...|[C

## Execute 2 consultas utilizando o método filterVertices.

In [132]:
# Método filterVertices  1)
population = graph.vertices.filter("population > 100000").show()

+--------------+---------+---------+----------+
|            id| latitude|longitude|population|
+--------------+---------+---------+----------+
|   João Pessoa|52.379189| 4.899431|    809015|
|Campina Grande|52.092876| 5.104480|    334176|
|         Patos|52.078663| 4.288788|    107605|
+--------------+---------+---------+----------+



In [133]:
# Execute 2 consultas utilizando o método filterVertices.

# Método filterVertices  2)
graph.vertices.filter("id = 'Paraíba'").show()

+---+--------+---------+----------+
| id|latitude|longitude|population|
+---+--------+---------+----------+
+---+--------+---------+----------+



### Não retornou nada, pois Paraíba não está na coluna "id"

In [134]:
# Método filterVertices 3)
graph.vertices.filter("latitude > 51").show()

+--------------+---------+---------+----------+
|            id| latitude|longitude|population|
+--------------+---------+---------+----------+
|   João Pessoa|52.379189| 4.899431|    809015|
|Campina Grande|52.092876| 5.104480|    334176|
|         Patos|52.078663| 4.288788|    107605|
|      Cabedelo| 53.61239| -0.22219|      9642|
|     São Bento| 53.52285| -1.13116|     34031|
|    Itaporanga| 52.05917|  1.15545|     24692|
|    São Mamede| 52.01667|  4.70833|      7724|
+--------------+---------+---------+----------+



## Lista 6

## Determine o nó central através do grau.

In [4]:
degree = graph.degrees
degreein = graph.inDegrees
degreeout = graph.outDegrees

# Sem o parâmetro "how = 'left'", as cidades com valores "zero" em qualquer coluna, não aparecem.
(degree.join(degreein, "id", how = "left").join(degreeout, "id", how = "left").fillna(0).sort("Degree").show())

+--------------------+------+--------+---------+
|                  id|degree|inDegree|outDegree|
+--------------------+------+--------+---------+
|         Itaporanga |     1|       0|        1|
|             Triunfo|     2|       1|        1|
|           São Bento|     3|       2|        1|
|           Rio Tinto|     3|       2|        1|
|               Areia|     3|       2|        1|
|          Mamanguape|     3|       1|        2|
|          São Mamede|     4|       2|        2|
|            Cabedelo|     4|       4|        0|
|      Campina Grande|     4|       3|        1|
|São José do rio d...|     5|       3|        2|
|          Itaporanga|     5|       1|        4|
|               Patos|     5|       2|        3|
|         João Pessoa|     6|       1|        5|
+--------------------+------+--------+---------+



#### O nó central através do grau foi João Pessoa

## Determine o nó central por centralidade utilizando a distância de Wasserman e a distância harmônica (consultar material).

#### Como o NetworkX não estava conseguindo ler o CSV, descobri que era problema do tipo do grafo que eu fiz e o nome das colunas que eu estava utilizando, que no caso eram: "src" e "dst".

In [5]:
nX = pd.read_csv('relationship_networkX.csv')

In [6]:
Graphtype = nx.DiGraph()

In [7]:
# KeyError: 'source'
# During handling of the above exception, another exception occurred:

# Quando rodei obtive esse erro, pois a minha coluna precisava estar com o nome de "source"

graph_nX = nx.from_pandas_edgelist(nX, create_using = Graphtype)

In [17]:
def run(acc, param, return_max_only = True):
    des = acc(param)
    
    print(json.dumps(des, indent = 2))
    return des if (return_max_only == False)else max(des, key = des.get)

### PS: Alguns nomes como João Pessoa, São Bento, São Mamede e São José do rio do Peixe, saíram errados, por causa do acento que usei na base de dados.

In [26]:
run(nx.harmonic_centrality, graph_nX)

{
  "Jo\u00e3o Pessoa": 3.6761904761904765,
  "Campina Grande": 5.283333333333333,
  "Patos": 4.426190476190477,
  "Cabedelo": 6.949999999999999,
  "S\u00e3o Bento": 4.7,
  "S\u00e3o Mamede": 4.533333333333334,
  "Triunfo": 3.6261904761904766,
  "Mamanguape": 3.233333333333333,
  "Areia": 4.2666666666666675,
  "Itaporanga": 3.7833333333333337,
  "Rio Tinto": 4.783333333333333,
  "S\u00e3o Jos\u00e9 do rio do Peixe": 5.616666666666666,
  "Itaporanga ": 0
}


'Cabedelo'

In [27]:
run(nx.closeness_centrality, graph_nX)

{
  "Jo\u00e3o Pessoa": 0.22522522522522526,
  "Campina Grande": 0.2976190476190476,
  "Patos": 0.2450980392156863,
  "Cabedelo": 0.42857142857142855,
  "S\u00e3o Bento": 0.2976190476190476,
  "S\u00e3o Mamede": 0.26881720430107525,
  "Triunfo": 0.21929824561403508,
  "Mamanguape": 0.1984126984126984,
  "Areia": 0.25252525252525254,
  "Itaporanga": 0.2450980392156863,
  "Rio Tinto": 0.30864197530864196,
  "S\u00e3o Jos\u00e9 do rio do Peixe": 0.34722222222222227,
  "Itaporanga ": 0.0
}


'Cabedelo'

## Determine o nó central por intermediação.

In [29]:
# Gtype = nx.Graph()
# G = nx.from_pandas_edgelist(node, create_using=Gtype)
# G2 = nx.from_pandas_edgelist(relationship, create_using=Gtype)

# Usando Betweenness
run(nx.betweenness_centrality, graph_nX)

{
  "Jo\u00e3o Pessoa": 0.26515151515151514,
  "Campina Grande": 0.25757575757575757,
  "Patos": 0.14393939393939395,
  "Cabedelo": 0.0,
  "S\u00e3o Bento": 0.09090909090909091,
  "S\u00e3o Mamede": 0.16666666666666669,
  "Triunfo": 0.14393939393939395,
  "Mamanguape": 0.11363636363636365,
  "Areia": 0.10606060606060606,
  "Itaporanga": 0.3106060606060606,
  "Rio Tinto": 0.30303030303030304,
  "S\u00e3o Jos\u00e9 do rio do Peixe": 0.08333333333333334,
  "Itaporanga ": 0.0
}


'Itaporanga'